In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

test.csv
train.csv



In [2]:
train = pd.read_csv('../input/train.csv')
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, pd.get_dummies(train['label']),
                                                                        test_size = 0.15,
                                                                        stratify=train['label'],
                                                                        random_state=42)

In [5]:
X_train = X_train.drop('label', axis = 1)
X_test = X_test.drop('label', axis=1)



In [6]:
X_train.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
31479,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38614,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27456,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30274,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
X_train = X_train.as_matrix()
X_test = X_test.as_matrix()
y_train = y_train.as_matrix()
y_test = y_test.as_matrix()

In [8]:

X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')

In [9]:
print(np.min(X_train))
print(np.max(X_train))
print(np.mean(X_train))
print(np.std(X_train))

print(np.min(X_test))
print(np.max(X_test))
print(np.mean(X_test))
print(np.std(X_test))

0.0
255.0
33.434
78.7047
0.0
255.0
33.267
78.5252


In [10]:
X_train = (X_train - 33.434) / 78.7047
X_test = (X_test - 33.434) / 78.7047

In [11]:
print(np.min(X_train))
print(np.max(X_train))
print(np.mean(X_train))
print(np.std(X_train))

print(np.min(X_test))
print(np.max(X_test))
print(np.mean(X_test))
print(np.std(X_test))

-0.424803
2.81516
-7.79016e-07
0.999997
-0.424803
2.81516
-0.00212163
0.997719


In [12]:
X_train[0]

array([[[-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308]],

       [[-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [-0.42480308],
        [

In [13]:
from keras.layers import Conv2D, Input, Dense, SpatialDropout2D, Flatten
from keras.models import Model

input_layer = Input(shape=(28,28, 1))
conv_layer = Conv2D(filters = 64, kernel_size = (3,3), strides=(1, 1), padding='valid')(input_layer)
dropout = SpatialDropout2D(0.5)(conv_layer)
dense_layer = Dense(64, activation = 'relu')(dropout)
flatten = Flatten()(dense_layer)
output = Dense(10, activation = 'softmax')(flatten)
model = Model([input_layer], output)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
model.summary()

Using TensorFlow backend.
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
spatial_dropout2d_1 (Spatial (None, 26, 26, 64)        0         
_________________________________________________________________
dense_1 (Dense)              (None, 26, 26, 64)        4160      
_________________________________________________________________
flatten_1 (Flatten)          (None, 43264)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                432650    
Total params: 437,450
Trainable params: 437,450
Non-trainable params: 0
_________________________________________________________________


In [14]:
#didn't divde by 255
model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data = (X_test, y_test))

Train on 35700 samples, validate on 6300 samples
Epoch 1/10
35700/35700 [==============================] - 92s - loss: 0.3184 - acc: 0.9062 - val_loss: 0.1338 - val_acc: 0.9616
Epoch 2/10
35700/35700 [==============================] - 92s - loss: 0.1240 - acc: 0.9629 - val_loss: 0.1131 - val_acc: 0.9643
Epoch 3/10
35700/35700 [==============================] - 93s - loss: 0.0820 - acc: 0.9745 - val_loss: 0.1040 - val_acc: 0.9695
Epoch 4/10
35700/35700 [==============================] - 92s - loss: 0.0643 - acc: 0.9795 - val_loss: 0.0875 - val_acc: 0.9733
Epoch 5/10
35700/35700 [==============================] - 93s - loss: 0.0530 - acc: 0.9832 - val_loss: 0.0862 - val_acc: 0.9767
Epoch 6/10
35700/35700 [==============================] - 92s - loss: 0.0441 - acc: 0.9856 - val_loss: 0.0912 - val_acc: 0.9754
Epoch 7/10
35700/35700 [==============================] - 92s - loss: 0.0381 - acc: 0.9867 - val_loss: 0.1019 - val_acc: 0.9743
Epoch 8/10
35700/35700 [==============================]

In [15]:
X_sub = pd.read_csv('../input/test.csv')
X_sub.head()




,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
X_sub = X_sub.as_matrix()


In [17]:
X_sub = X_sub.reshape(X_sub.shape[0], 28, 28, 1).astype('float32')


In [18]:
X_sub = (X_sub - 33.434) / 78.7047




In [19]:
print(np.min(X_sub))
print(np.max(X_sub))
print(np.mean(X_sub))
print(np.std(X_sub))

-0.424803
2.81516
-0.00104776
0.998917


In [20]:
predictions = model.predict(X_sub, verbose=0)

In [21]:
# select the indix with the maximum probability
results = np.argmax(predictions,axis = 1)

results = pd.Series(results,name="Label")
results.head()

0    2
1    0
2    9
3    0
4    3
Name: Label, dtype: int64

In [22]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("cnn_mnist_datagen.csv",index=False)